 # 統計的仮説検定

In [1]:
# %
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
import unittest
import doctest
from matplotlib import rcParams
import os
import warnings
warnings.simplefilter('ignore', FutureWarning)
path = os.path.dirname(os.path.abspath(__file__))

 ## 母平均に関する1標本の検定

 ### 統計的仮説検定の初歩

 #### 母平均に関する1標本のt検定

 #### 帰無仮説・対立仮設

 #### 有意差

 #### t検定の直感的な考え方

 #### 平均値の差が大きいだけでは有意差は得られない

 #### 検定統計量

 #### t値の復習

 #### ここまでのまとめその１

 #### 第一種の過誤・第二種の過誤

 #### 有意水準

 #### 棄却域・受容域

 #### p値

 #### ここまでのまとめその２

 #### t値とt分布の関係の復習

 #### 片側検定・両側検定

 #### 棄却域の計算方法

 #### p値の計算方法

 #### 数式を使ったまとめ

 #### 分析の準備

In [2]:
junk_food = pd.read_csv(f'{path}/data/6-1-1-junk-food-weight.csv')['weight']
junk_food.head()

0    58.529820
1    52.353039
2    74.446169
3    52.983263
4    55.876879
Name: weight, dtype: float64

 #### t値の計算

In [3]:
x_bar = np.mean(junk_food)
round(x_bar, 3)

n = len(junk_food)
df = n - 1
df

u = np.std(junk_food, ddof=1)
se = u / np.sqrt(n)
round(se, 3)

t_sample = (x_bar - 50) / se
round(t_sample, 3)

2.75

 #### 棄却域の計算

In [4]:
round(stats.t.ppf(q=0.025, df=df), 3)

-2.093

 #### p値の計算

In [5]:
p_value = stats.t.cdf(x=-np.abs(t_sample), df=df) * 2
round(p_value, 3)

stats.ttest_1samp(junk_food, popmean=50)

TtestResult(statistic=2.7503396831713434, pvalue=0.012725590012524155, df=19)

 #### シミュレーションによるp値の計算

In [6]:
n = len(junk_food)
u = np.std(junk_food, ddof=1)
t_value_array = np.zeros(50000)

np.random.seed(1)
norm_dist = stats.norm(loc=50, scale=u)
for i in range(0, 50000):
    # 標本の抽出
    sample = norm_dist.rvs(size=n)
    # t値の計算
    sample_mean = np.mean(sample)  # 標本平均
    sample_std = np.std(sample, ddof=1)  # 標本標準偏差
    sample_se = sample_std / np.sqrt(n)  # 標本平均の標準誤差
    t_value_array[i] = (sample_mean - 50) / sample_se  # t値

p_sim = (sum(t_value_array >= t_sample) / 50000) * 2
round(p_sim, 3)

0.013

 ### 平均値の差の検定

 #### 2群のデータに対するt検定

 #### 対応のあるt検定

 #### 分析の準備

In [7]:
paired_test_data = pd.read_csv(f'{path}/data/6-2-1-paired-t-test.csv')
print(paired_test_data)

  person medicine  body_temperature
0      A   before              36.2
1      B   before              36.2
2      C   before              35.3
3      D   before              36.1
4      E   before              36.1
5      A    after              36.8
6      B    after              36.1
7      C    after              36.8
8      D    after              37.1
9      E    after              36.9


 #### 対応のあるt検定

In [8]:
# 薬を飲む前と飲んだ後の標本平均
before = paired_test_data.query('medicine == "before"')['body_temperature']
after = paired_test_data.query('medicine == "after"')['body_temperature']
# アレイに変換
before = np.array(before)
after = np.array(after)
# 差を計算
diff = after - before
diff

stats.ttest_1samp(diff, popmean=0)

stats.ttest_rel(after, before)

TtestResult(statistic=2.901693483620596, pvalue=0.044043109730074276, df=4)

 #### 対応のないt検定（不等分散）

 #### 対応のないt検定（等分散）

In [9]:
# 平均値
x_bar_bef = np.mean(before)
x_bar_aft = np.mean(after)

# 分散
u2_bef = np.var(before, ddof=1)
u2_aft = np.var(after, ddof=1)

# サンプルサイズ
m = len(before)
n = len(after)

# t値
t_value = (x_bar_aft - x_bar_bef) / np.sqrt((u2_bef / m) + (u2_aft / n))
round(t_value, 3)

df = (u2_bef / m + u2_aft / n)**2 / ((u2_bef / m)
                                     ** 2 / (m - 1) + (u2_aft / n)**2 / (n - 1))
round(df, 3)

p_value = stats.t.cdf(-np.abs(t_value), df=df) * 2
round(p_value, 5)

stats.ttest_ind(after, before, equal_var=False)

Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)

 #### pハッキング

 ### 分割表の検定

 #### 分割表を用いるメリット

 #### 本章で扱う例題

 #### 期待度数との差を求める
 $$
 \chi^2 = \sum_{i=1}^r \sum_{j=1}^c \frac{(O_{ij} - E_{ij})^2}{E_{ij}}

 #### 分析の準備

 #### p値の計算

In [10]:
1 - stats.chi2.cdf(x=6.667, df=1)

0.009821437357809604

 #### 分割表の検定

In [11]:
click_data = pd.read_csv(f'{path}/data/6-3-1-click_data.csv')
print(click_data)

cross = pd.pivot_table(
    data=click_data,
    values='freq',
    aggfunc='sum',
    index='color',
    columns='click'
)
print(cross)

stats.chi2_contingency(cross, correction=False)

  color  click  freq
0  blue  click    20
1  blue    not   230
2   red  click    10
3   red    not    40
click  click  not
color            
blue      20  230
red       10   40


Chi2ContingencyResult(statistic=6.666666666666666, pvalue=0.009823274507519247, dof=1, expected_freq=array([[ 25., 225.],
       [  5.,  45.]]))

 ### 検定の結果の解釈

 #### p値が0.05以下だったときの結果の書き方

 #### p値が0.05より大きかったときの結果の書き方

 #### 仮説検定における、よくある間違い

 #### p値が小さくても、差が大きいとは限らない

 #### p値が0.05より大きくても差がないとは言えない

 #### 検定の非対称性

 #### 有意水準は、検定をする前に決めておく

 #### 統計的仮説検定は必要か

 #### 仮説は正しいか

In [12]:
unittest.main(argv=[''], verbosity=2, exit=False)
doctest.testmod(verbose=True)

3 items had no tests:
    __main__
    __main__.__VSCODE_compute_hash
    __main__.__VSCODE_wrap_run_cell
0 tests in 3 items.
0 passed and 0 failed.
Test passed.



----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


TestResults(failed=0, attempted=0)